# Setup Instructions for Ubuntu 22.10

Run this on your local machine but change the external IP address and the PEM file with the data viven in the reservation email from the TechZone

In [ ]:
# LOGIN CELL
export EXTERNAL_IPADDR=159.122.199.26
export PEM_FILE=ub.pem
chmod 600 $PEM_FILE

ssh itzuser@$EXTERNAL_IPADDR -p 2223 -i $PEM_FILE

Once you are on the lab client, run this command

In [ ]:
# Clone the workshop repository
git clone https://github.com/angel-ibm/deployment-databand.git

And now, you have two options:

1. run a single script that installs all necessary packages
2. run every single command step-by-step (only necessary if something goes wrong)

## 1. single install script

In [ ]:
# Run this to install automaticall all necessary packages
cd ~/deployment-databand/client
./ubuntu.sh

## 2. Step-by-step installation

You may want to run the following steps if something went wrong with the single install script or if you just want to follow the process

In [ ]:
# Create an alias for python3
sudo apt update -y
sudo apt upgrade -y

echo "alias python=python3" >> .bash_aliases
. ./.bash_aliases 

In [ ]:
# Install pip
sudo apt install python3-pip -y

In [ ]:
# Install Jupyter Notebooks
sudo apt install jupyter -y
pip install --upgrade jupyter_core
pip install jupyterlab
pip install bash_kernel
python -m bash_kernel.install
pip install markupsafe==2.0.1
. ./.profile

In [ ]:
# Install snap
sudo snap install yq

# Install helm
sudo snap install helm --classic

# Install oc 
wget https://mirror.openshift.com/pub/openshift-v4/x86_64/clients/ocp/stable-4.10/openshift-client-linux-4.10.54.tar.gz
tar -zxf openshift-client-linux-4.10.54.tar.gz
sudo mv kubectl oc /usr/local/bin
rm README.md openshift-client-linux-4.10.54.tar.gz

In [ ]:
# Install docker
sudo apt-get install ca-certificates curl gnupg
sudo mkdir -m 0755 -p /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
echo \
  "deb [arch="$(dpkg --print-architecture)" signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu \
  "$(. /etc/os-release && echo "$VERSION_CODENAME")" stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update
sudo apt-get install docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin -y
sudo usermod -aG docker $USER

In [ ]:
# Start Jupyter

jupyter lab --ip 0.0.0.0 --port 8080 --notebook-dir ~/deployment-databand